## Hands-on

### Sentiment Analysis Using Naïve Bayes

In [3]:
#importing necessary libraries
import sys
import os
import re
import csv
from operator import itemgetter
import unicodedata

#importing classes
from codes.proc import Proc as P

### Phase 1
***
- ##### Data colect

In [4]:
filename = 'ODemonioeaSrtaPrym.pdf'
p = P(filename)

p.convertPDF()

name1 = 'Prym'
name2 = 'estrangeiro'

#Variable 'relation' is used to build training dataset
#relation = 'afinidade'
txt = p.get_text()
p.fname(txt,name1,name2)

txtbooks/ODemonioeaSrtaPrym.txt
estrangeiro para si mesmo, lembrandose das palavras da senhorita Prym


------------------------


Prym podia se espalhar, afugentando caçadores e turistas, ele convocara uma reunião
de emergência. Naquele momento, enquanto Chantal se dirigia para a floresta, o estrangeiro


------------------------


estrangeiro havia providenciado tudo, assinado os papéis de transferência
do metal, providenciado que o mesmo fosse vendido e o dinheiro depositado na conta recémaberta da senhorita Prym


------------------------




***
- ##### Preprocess

In this process we check if the `dataset of training` already exists, if not gets the `training base` and applies preprocess steps and generating `preprocessed-tb`.

In [5]:
sw = []
with open('stopwords.txt') as f:
    sw = f.read().splitlines()

In [6]:
#look for stopwords
def common_elements(l1,l2):
    l = []
    for x in l1:
        if x in l2:
            l.append(x)
    return l

In [7]:
def preprocess(text):
    "remove accents"
    data = text
    normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
    text = (str(normal).replace('b\'','').replace('\'',''))

    "remove punctuation"
    text = ''.join(c for c in text if c not in punctuation)
    #row[1] = row[1].replace('-','')

    "lower case"
    lrow = text.split()
    lrow = [x.lower() for x in lrow]
    #lrow.pop(0) # remove primeiro elemento (nome)
    
    #remove stopwords
    common = common_elements(sw,lrow)
    if len(common) > 0:
        for x in common:
            if x in lrow:
                lrow.remove(x)

    line = ' '.join(str(x) for x in lrow)

    # remove digits
    remove_digits = str.maketrans('', '', digits)
    line = line.translate(remove_digits)

    return line

In [8]:
#Training Base
tb = 'tb1.csv'
# Baseline dataset
base = 'base11.csv'

if not os.path.exists('./preprocessed-tb.csv'):
    with open('preprocessed-tb.csv', 'w', newline='') as outputfile:
        with open(tb, newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='"')
            writer = csv.writer(outputfile, delimiter=',', quotechar='"')

            for row in reader:
                row[1] = preprocess(row[1])
                writer.writerow(row)
else:
    tb = 'preprocessed-tb.csv'

***
#### Load data


In [9]:
c = [] # [(class, occurences)]
classes = [] # classes names
tab = []
prob = []
P1 = []
P2 = []
n = 0

In [10]:
with open(tb, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in spamreader:
        n = n+1
        tab.append(row)
        if row[0] not in classes:
            classes.append(row[0])
            c.append((row[0],1))
        else:
            for (i, j) in enumerate(c):
                if(j[0] == row[0]):
                    c[i] = ((row[0],c[i][1] + 1))


***
***
### Phase 2: Method

- ##### Prior Probability

In [11]:
print('Prior Probability')
if '' in classes:
    classes.remove('')

for x in classes:
    x = re.sub("[^a-zA-Z|^\s|^\t|^\r]+", "", x)
    for i in c:
        if i[0] == x:
            pi = i[1]/n
            break
        else:
            pi = 0
    P1.append((x,pi))
    
print(P1)

Prior Probability
[]


- ##### Constructing the Unigram Language Model & Posterior Probability

In [ ]:
bayes = []
count = 0
with open(base, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for e in spamreader:
        count = count +1
        entry = preprocess(e[0])
        at = list(set(entry.split()))
        for c in classes:
            for a in at:
                num = 0
                for var in tab:
                    if c == var[0] and a in var[1]:
                        num = num + var[1].count(a)

                num_at = 0
                for a2 in at:
                    for var in tab:
                        if c == var[0] and a2 in var[1]:
                            num_at = num_at + var[1].count(a2)

                    num_at = num_at  + 1

                pi = (num +1)/ num_at
                prob.append((c,a,pi))

        for c in classes:
            pi = 1
            for e in entry.split():
                for p in prob:
                    if c == p[0] and e == p[1]:
                        pi = (pi*10) * (p[2]*10)                       

            P2.append((c,pi))


    print('Posterior Probability\n')
    
    #Bayes' Rule

    for p1 in P1:
        for p2 in P2:
            if p1[0] is p2[0]:
                p = p1[1] * p2[1]
                                
                bayes.append((p1[0],p))

    print('\n---\n')
    f = 0
    nf = 0

    for b in bayes:
        if b[0] == 'afinidade':
            f = f+b[1]
        else:
            nf = nf+b[1]

***
- ##### Results

In [13]:
if f > nf:
    nv_class = 'afinidade'
else:
    nv_class = 'inimizade'

half_list = int((len(bayes)/2))
half_left_list = bayes[half_list:]
half_right_list = bayes[:half_list]

#print(hal,half_right_list)
x = half_left_list[0][0]
y = half_right_list[0][0]
count_x = 0
count_y = 0
neutral = 0

for i,j in zip(half_left_list,half_right_list):
    if i[1] > j[1]:
        count_x = count_x + 1
    elif i[1] < j[1]:
        count_y = count_y + 1
    else:
        neutral = neutral + 1


print('Classification:', nv_class)
print(x+':', count_x, y+':', count_y)
print(f,nf)
print('f->',((f*100)/(nf+f)), 'nf->',(nf*100)/(nf+f))


NameError: name 'nf' is not defined